In [1]:
from ortools.linear_solver import pywraplp
import psutil

In [1]:
def read_data_from_file(filename):
    """
    Reads data from a file with the specified format for an air traffic scheduling problem.

    Args:
        filename (str): The path to the data file.

    Returns:
        tuple: A tuple containing the parsed data:
            - num_planes (int): The number of planes.
            - freeze_time (int): The freeze time.
            - planes_data (list): A list of dictionaries, where each dictionary contains
              the data for a plane.
            - separation_times (list of lists): A 2D list of separation times.
    """
    try:
        with open(filename, "r") as f:
            # Read the first line: number of planes and freeze time
            first_line = f.readline().strip().split()
            num_planes = int(first_line[0])
            freeze_time = int(first_line[1])

            planes_data = []
            separation_times = []

            for _ in range(num_planes):
                line = f.readline().strip().split()
                appearance_time = int(line[0])
                earliest_landing_time = int(line[1])
                target_landing_time = int(line[2])
                latest_landing_time = int(line[3])
                penalty_early = float(line[4])
                penalty_late = float(line[5])
                planes_data.append(
                    {
                        "appearance_time": appearance_time,
                        "earliest_landing_time": earliest_landing_time,
                        "target_landing_time": target_landing_time,
                        "latest_landing_time": latest_landing_time,
                        "penalty_early": penalty_early,
                        "penalty_late": penalty_late,
                    }
                )

                separation_row = []
                while len(separation_row) < num_planes:
                    line = f.readline().strip().split()
                    separation_row.extend([int(x) for x in line])

                separation_times.append(separation_row)

        return num_planes, freeze_time, planes_data, separation_times

    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None, None, None, None
    except ValueError:
        print(f"Error: Error reading data in file '{filename}'.")
        return None, None, None, None

In [ ]:
filename = "data/airland8.txt"

num_planes, freeze_time, planes_data, separation_times = read_data_from_file(filename)

print("Number of planes:", num_planes)
print("Freeze time:", freeze_time)
print("\nPlane data:")
for i, plane in enumerate(planes_data):
    print(f"Plane {i+1}: {plane}")
print("\nSeparation times:")
for i, row in enumerate(separation_times):
    print(f"After plane {i+1}: {row}")

In [4]:
def multiple_runways_problem(
    num_planes, planes_data, separation_times, num_runways, objective="cost"
):
    # Create the LP solver
    solver = pywraplp.Solver.CreateSolver("SAT")  # Using the SAT solver
    variables = {}

    # Decision Variables
    # x_i: Landing time for plane i
    # (1)
    landing_times = [
        solver.NumVar(
            planes_data[i]["earliest_landing_time"],
            planes_data[i]["latest_landing_time"],
            f"LandingTime_{i}",
        )
        for i in range(num_planes)
    ]
    variables["landing_times"] = landing_times

    # delta_ij:  Fraction representing if plane i lands before plane j (0 to 1)
    # Note: In a pure LP model, we relax the integrality constraint.
    landing_order = {}
    for i in range(num_planes):
        for j in range(num_planes):
            if i != j:
                landing_order[(i, j)] = solver.NumVar(0, 1, f"LandingOrder_{i}_{j}")
    variables["landing_order"] = landing_order

    # alpha_i: Time by which plane i lands before its target time
    early_deviation = [
        solver.NumVar(
            0,
            max(
                planes_data[i]["target_landing_time"]
                - planes_data[i]["earliest_landing_time"],
                0,
            ),
            f"EarlyDeviation_{i}",
        )
        for i in range(num_planes)
    ]
    variables["early_deviation"] = early_deviation

    # beta_i: Time by which plane i lands after its target time
    late_deviation = [
        solver.NumVar(
            0,
            max(
                planes_data[i]["latest_landing_time"]
                - planes_data[i]["target_landing_time"],
                0,
            ),
            f"LateDeviation_{i}",
        )
        for i in range(num_planes)
    ]
    variables["late_deviation"] = late_deviation

    # z_ij: 1 if plane i and plane j land on the same runway, 0 otherwise
    same_runway = {}
    for i in range(num_planes):
        for j in range(num_planes):
            if i != j:
                same_runway[(i, j)] = solver.NumVar(0, 1, f"SameRunway_{i}_{j}")

    variables["same_runway"] = same_runway

    # y_ir: 1 if plane i lands on runway r, 0 otherwise
    landing_runway = {}
    for i in range(num_planes):
        for r in range(num_runways):
            landing_runway[(i, r)] = solver.NumVar(0, 1, f"LandingRunway_{i}_{r}")

    variables["landing_runway"] = landing_runway

    # Constraints
    # (2)
    for i in range(num_planes):
        for j in range(i + 1, num_planes):
            solver.Add(landing_order[(i, j)] + landing_order[(j, i)] == 1)

    # Set W
    # (3)
    certain_with_separation_pairs = []
    for i in range(num_planes):
        for j in range(num_planes):
            if i != j:
                earliest_i, latest_i = (
                    planes_data[i]["earliest_landing_time"],
                    planes_data[i]["latest_landing_time"],
                )
                earliest_j = planes_data[j]["earliest_landing_time"]
                separation_ij = separation_times[i][j]
                if latest_i < earliest_j and latest_i + separation_ij <= earliest_j:
                    certain_with_separation_pairs.append((i, j))

    # Set V
    # (4)
    certain_with_no_separation_pairs = []
    for i in range(num_planes):
        for j in range(num_planes):
            if i != j:
                earliest_i, latest_i = (
                    planes_data[i]["earliest_landing_time"],
                    planes_data[i]["latest_landing_time"],
                )
                earliest_j = planes_data[j]["earliest_landing_time"]
                separation_ij = separation_times[i][j]
                if latest_i < earliest_j and latest_i + separation_ij > earliest_j:
                    certain_with_no_separation_pairs.append((i, j))

    # Set U
    # (5)
    uncertain_pairs = []
    for i in range(num_planes):
        for j in range(num_planes):
            if i != j:
                earliest_i, latest_i = (
                    planes_data[i]["earliest_landing_time"],
                    planes_data[i]["latest_landing_time"],
                )
                earliest_j, latest_j = (
                    planes_data[j]["earliest_landing_time"],
                    planes_data[j]["latest_landing_time"],
                )

                if (
                    (earliest_j <= earliest_i <= latest_j)
                    or (earliest_j <= latest_i <= latest_j)
                ) or (
                    (earliest_i <= earliest_j <= latest_i)
                    or (earliest_i <= latest_j <= latest_i)
                ):
                    uncertain_pairs.append((i, j))

    # Enforce separation for pairs where order is determined (Set V)
    # (6) and (7)
    for i, j in certain_with_no_separation_pairs:
        solver.Add(landing_order[(i, j)] == 1)  # (6)
        solver.Add(
            landing_times[j]
            >= landing_times[i] + separation_times[i][j] * same_runway[(i, j)]
        )  # (7)

    # Enforce order for pairs where order is determined and separation is automatic (Set W)
    # (6)
    for i, j in certain_with_separation_pairs:
        solver.Add(landing_order[(i, j)] == 1)  # (6)

    M = sum(plane["latest_landing_time"] for plane in planes_data)

    for i, j in uncertain_pairs:
        # solver.Add(landing_times[j] >= landing_times[i] + separation_times[i][j] * same_runway[(i, j)] + separation_times_between_runways[(i, j)] * (1 - same_runway[(i, j)]) - M * (landing_order[(j, i)])) # (8)
        latest_i = planes_data[i]["latest_landing_time"]
        earliest_j = planes_data[j]["earliest_landing_time"]

        solver.Add(
            landing_times[j]
            >= landing_times[i]
            + separation_times[i][j] * same_runway[(i, j)]
            - (latest_i + separation_times[i][j] - earliest_j) * (landing_order[(j, i)])
        )  # (8)

    # 4. Relating Deviation Variables to Landing Times
    for i in range(num_planes):
        earliest_i = planes_data[i]["earliest_landing_time"]
        latest_i = planes_data[i]["latest_landing_time"]
        target_i = planes_data[i]["target_landing_time"]

        # (14)
        solver.Add(early_deviation[i] >= target_i - landing_times[i])

        # (15)
        solver.Add(early_deviation[i] >= 0)
        solver.Add(early_deviation[i] <= target_i - earliest_i)

        # (16)
        solver.Add(late_deviation[i] >= landing_times[i] - target_i)

        # (17)
        solver.Add(late_deviation[i] >= 0)
        solver.Add(late_deviation[i] <= latest_i - target_i)

        # (18)
        solver.Add(
            landing_times[i] == target_i - early_deviation[i] + late_deviation[i]
        )

    # New constraints for multiple runways
    # (28)
    for i in range(num_planes):
        solver.Add(solver.Sum(landing_runway[(i, r)] for r in range(num_runways)) == 1)

    # (29)
    for i in range(num_planes):
        for j in range(i + 1, num_planes):
            solver.Add(same_runway[(i, j)] == same_runway[(j, i)])

    # (30)
    for i in range(num_planes):
        for j in range(i + 1, num_planes):
            for r in range(num_runways):
                solver.Add(
                    same_runway[(i, j)]
                    >= landing_runway[(i, r)] + landing_runway[(j, r)] - 1
                )

    if objective == "cost":
        objective = solver.Objective()

        for i in range(num_planes):
            objective.SetCoefficient(early_deviation[i], planes_data[i]["penalty_early"])
            objective.SetCoefficient(late_deviation[i], planes_data[i]["penalty_late"])

        objective.SetMinimization()
    
    if objective == "balance_workload":
        # Workload for each runway
        workload = [solver.IntVar(0, num_planes, f"Workload_{r}") for r in range(num_runways)]

        # Relate workload to landing_runway variables
        for r in range(num_runways):
            solver.Add(workload[r] == solver.Sum(landing_runway[(i, r)] for i in range(num_planes)))

        # Variables for max and min workload
        max_workload = solver.IntVar(0, num_planes, "MaxWorkload")
        min_workload = solver.IntVar(0, num_planes, "MinWorkload")

        # Define max_workload and min_workload
        for r in range(num_runways):
            solver.Add(max_workload >= workload[r])
            solver.Add(min_workload <= workload[r])

        # Minimize the difference between max and min workload
        objective = solver.Objective()
        objective.SetCoefficient(max_workload, 1)
        objective.SetCoefficient(min_workload, -1)
        objective.SetMinimization()
        
    if objective == "combined":
        # Weights for the combined objective
        weight_cost = 1.0  # Adjust as needed
        weight_workload = 100  # Adjust as needed

        # Workload for each runway
        workload = [solver.Sum(landing_runway[(i, r)] for i in range(num_planes)) for r in range(num_runways)]

        # Variables for max and min workload
        max_workload = solver.NumVar(0, num_planes, "MaxWorkload")
        min_workload = solver.NumVar(0, num_planes, "MinWorkload")

        # Define max_workload and min_workload
        for r in range(num_runways):
            solver.Add(max_workload >= workload[r])
            solver.Add(min_workload <= workload[r])

        # Define the combined objective function
        objective = solver.Objective()

        # Add the cost component
        for i in range(num_planes):
            objective.SetCoefficient(early_deviation[i], weight_cost * planes_data[i]["penalty_early"])
            objective.SetCoefficient(late_deviation[i], weight_cost * planes_data[i]["penalty_late"])

        # Add workload imbalance components
        objective.SetCoefficient(max_workload, weight_workload)
        objective.SetCoefficient(min_workload, -weight_workload)

        # Set the optimization direction
        objective.SetMinimization()


    return solver, variables

## Evaluation Metrics 

In [5]:
def calculate_execution_time(solver):
    """
    Calculates the total execution time of the solver.
    
    Args:
        solver: The OR-Tools solver instance.
        
    Returns:
        float: Execution time in seconds.
    """
    return solver.WallTime() / 1000  # Convert milliseconds to seconds

def calculate_num_variables(solver):
    """
    Calculates the total number of variables in the model.
    
    Args:
        solver: The OR-Tools solver instance.
    
    Returns:
        int: The total number of variables.
    """
    return solver.NumVariables()

def calculate_num_constraints(solver):
    """
    Calculates the total number of constraints in the model.
    
    Args:
        solver: The OR-Tools solver instance.
    
    Returns:
        int: The total number of constraints.
    """
    return solver.NumConstraints()

def calculate_runway_workload(variables, num_runways, num_planes):
    """
    Calculates the workload (number of landings) for each runway.

    Args:
        variables: Dictionary containing model variables.
        num_runways: Number of runways.
        num_planes: Number of planes.

    Returns:
        list: Workload for each runway.
    """
    workloads = [0] * num_runways
    for r in range(num_runways):
        workloads[r] = sum(
            variables["landing_runway"][(i, r)].solution_value() for i in range(num_planes)
        )
    return workloads

def calculate_workload_imbalance(workloads):
    """
    Calculates the imbalance in workload across runways.

    Args:
        workloads: List of workloads for each runway.

    Returns:
        int: Difference between the maximum and minimum workload.
    """
    return max(workloads) - min(workloads)

def calculate_total_penalty(variables, planes_data):
    """
    Calculates the total penalty of the solution based on early and late deviations.

    Args:
        variables: Dictionary containing model variables.
        planes_data: List of dictionaries with data for each plane.

    Returns:
        float: Total penalty of the solution.
    """
    total_penalty = 0
    for i, plane in enumerate(planes_data):
        total_penalty += (
            variables["early_deviation"][i].solution_value() * plane["penalty_early"]
            + variables["late_deviation"][i].solution_value() * plane["penalty_late"]
        )
    return total_penalty

def calculate_memory_usage():
    """
    Measures the memory usage of the solver process during execution.

    Returns:
        float: Memory usage in megabytes (MB).
    """
    process = psutil.Process()
    memory_in_bytes = process.memory_info().rss
    return memory_in_bytes / (1024 * 1024)  # Convert to MB

In [6]:
def summarize_metrics_MIP(solver, variables, num_planes, num_runways=None, planes_data=None):
    """
    Calculates and prints metrics specific to MIP problems.

    Args:
        solver: Instance of the OR-Tools solver.
        variables: Dictionary containing the model variables.
        num_planes: Number of planes in the problem.
        num_runways: (Optional) Number of runways.
        planes_data: (Optional) Data of planes used to calculate penalties.

    Returns:
        dict: Dictionary containing all calculated metrics.
    """
    print("=" * 60)
    print("METRICS FOR MIP PROBLEMS")
    print("=" * 60)

    metrics = {}

    # Execution time
    execution_time = calculate_execution_time(solver)
    metrics['execution_time'] = execution_time
    print(f"-> Execution time: {execution_time:.2f} seconds")

    # Number of variables
    num_variables = calculate_num_variables(solver)
    metrics['num_variables'] = num_variables
    print(f"-> Number of variables in the model: {num_variables}")

    # Number of constraints
    num_constraints = calculate_num_constraints(solver)
    metrics['num_constraints'] = num_constraints
    print(f"-> Number of constraints in the model: {num_constraints}")

    # Workload per runway (if applicable)
    if num_runways is not None:
        workloads = calculate_runway_workload(variables, num_runways, num_planes)
        workload_imbalance = calculate_workload_imbalance(workloads)
        metrics['workloads'] = workloads
        metrics['workload_imbalance'] = workload_imbalance
        print(f"-> Workload per runway: {workloads}")
        print(f"-> Workload imbalance: {workload_imbalance}")
    else:
        print("-> Workload per runway not calculated (number of runways not provided).")

    # Total penalty (if applicable)
    if planes_data is not None:
        total_penalty = calculate_total_penalty(variables, planes_data)
        metrics['total_penalty'] = total_penalty
        print(f"-> Total penalty: {total_penalty:.2f}")
    else:
        print("-> Total penalty not calculated (plane data not provided).")
    
    # Memory usage
    memory_usage = calculate_memory_usage()
    metrics['memory_usage'] = memory_usage
    print(f"-> Memory usage: {memory_usage:.2f} MB")
    
    print("=" * 60)
    return metrics


In [ ]:
solver, variables = multiple_runways_problem(
    num_planes, planes_data, separation_times, 1,
)
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print("Number of variables created:", solver.NumVariables())
    print("Number of constraints:", solver.NumConstraints())
    print("\nSolution for Balancing Workload:")
    print(f"Objective Value (Balance Workload): {solver.Objective().Value()}")
    
    metrics = summarize_metrics_MIP(
        solver,
        variables,
        num_planes,
        num_runways = 1,
        planes_data = planes_data
    )

    # Extract landing times
    landing_times = [
        variables["landing_times"][i].solution_value() for i in range(num_planes)
    ]

    # Calculate the optimal cost for the "cost" objective
    optimal_cost = 0
    for i in range(num_planes):
        target_time = planes_data[i]["target_landing_time"]
        penalty_early = planes_data[i]["penalty_early"]
        penalty_late = planes_data[i]["penalty_late"]

        early_deviation = max(0, target_time - landing_times[i])
        late_deviation = max(0, landing_times[i] - target_time)

        optimal_cost += penalty_early * early_deviation + penalty_late * late_deviation

    print("\nCalculated Optimal Cost (using 'cost' objective):", optimal_cost)

    print("\nOptimal Landing Times:")
    for i in range(num_planes):
        print(
            f"Plane {i+1}: {variables['landing_times'][i].solution_value()} | Target Time: {planes_data[i]['target_landing_time']}"
        )

    print("\nLanding Order Variables (Note: May not be strictly 0 or 1 in LP):")
    for i in range(num_planes):
        for j in range(num_planes):
            if i != j:
                print(
                    f"LandingOrder_{i}_{j}: {variables['landing_order'][(i, j)].solution_value()}"
                )

else :
    print("No optimal solution found.")

In [1]:
from ALS import solve_multiple_runways_mip
from ALS.utils import read_data

filename = "data/airland9.txt"

num_planes, planes_data, separation_times = read_data(filename)

solve_multiple_runways_mip(num_planes, planes_data, separation_times, num_runways=1)

---------- Reading data from airland9.txt ----------

Number of planes: 100 

---------- Creating MIP solver ----------

Number of decision variables created: 20200
Number of constraints: 22019

---------- Solving MIP ----------



I0000 00:00:1736767446.352260   95949 sigint.cc:31] ^C pressed 1 times. Interrupting the solver. Press 3 times to force termination.


No optimal solution found.
Best feasible solution found: 5691.55
